In [13]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from msrest.authentication import ApiKeyCredentials
import os
import requests

# Replace with your Custom Vision details
ENDPOINT = "https://beeml2.cognitiveservices.azure.com/"
TRAINING_KEY = "1c333c79890749298585bea11781f890"
PROJECT_ID = "10d0f2e6-03e6-4594-9433-7b6d4f3f451d"

# Authenticate
credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create output directory
output_dir = "exported_images"
os.makedirs(output_dir, exist_ok=True)

# Get all images with bounding box info
images = trainer.get_tagged_images(PROJECT_ID)

for img in images:
    img_url = img.original_image_uri
    img_id = img.id
    width = img.width
    height = img.height

    # Download the image
    img_data = requests.get(img_url).content
    with open(f"{output_dir}/{img_id}.jpg", "wb") as handler:
        handler.write(img_data)

    # Save labels in YOLO format (class x_center y_center width height)
    with open(f"{output_dir}/{img_id}.txt", "w") as label_file:
        for tag in img.tags:
            if tag.bounding_box:
                left = tag.bounding_box.left * width
                top = tag.bounding_box.top * height
                box_width = tag.bounding_box.width * width
                box_height = tag.bounding_box.height * height

                # Convert to YOLO format (normalized)
                x_center = (left + box_width / 2) / width
                y_center = (top + box_height / 2) / height
                w = box_width / width
                h = box_height / height

                label_file.write(f"{tag.tag_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print(f"Downloaded {img_id}.jpg with labels and bounding boxes.")

print("✅ Export complete!")


AttributeError: 'ImageTag' object has no attribute 'bounding_box'

In [19]:
for kw in img.regions:
    print(kw)

{'additional_properties': {}, 'region_id': '5ae7c513-e38a-4178-9177-699b29ef30eb', 'tag_name': 'bad_egg', 'created': datetime.datetime(2025, 3, 4, 22, 28, 11, 306495), 'tag_id': '62988574-c62f-47c6-b27a-b47c5eb5ab30', 'left': 0.19160928, 'top': 0.04231696, 'width': 0.32744378, 'height': 0.3069316}
{'additional_properties': {}, 'region_id': '68a128f8-ca49-44ef-ad3d-e4b20a9e62f5', 'tag_name': 'big_good_egg', 'created': datetime.datetime(2025, 3, 4, 22, 28, 11, 306495), 'tag_id': '2219cae2-3cb2-441b-bfad-30af40562073', 'left': 0.35835105, 'top': 0.2930338, 'width': 0.36180425, 'height': 0.32937708}
{'additional_properties': {}, 'region_id': 'b7a67193-08de-48df-a431-2d3b15a4ae1a', 'tag_name': 'good_egg', 'created': datetime.datetime(2025, 3, 4, 22, 50, 0, 874904), 'tag_id': 'a1c794a9-9313-41cc-abe8-4810c6a7794f', 'left': 0.0018590838, 'top': 0.0, 'width': 0.28583425, 'height': 0.15151855}


In [10]:
img.created

datetime.datetime(2025, 3, 4, 22, 27, 47, 148159)